LSTM FEATURE ABLATION - Testing Redundancy Hypothesis

Feature Sets to Test:
  returns_only        : 1 features - ['RET_1']
  returns_momentum    : 3 features - ['RET_1', 'RSI_14', 'MACD_HIST']
  momentum_original   : 5 features - ['RSI_14', 'MACD_HIST', 'RET_1', 'RET_5', 'RET_15']
  momentum_clean      : 3 features - ['RSI_14', 'MACD_HIST', 'RET_1']
  momentum_position   : 5 features - ['RSI_14', 'MACD_HIST', 'RET_1', 'BB_POSITION', 'PRICE_EMA21_DIST']
  minimal             : 2 features - ['RET_1', 'RSI_14']
  comprehensive       : 8 features - ['RSI_14', 'MACD_HIST', 'RET_1', 'BB_POSITION', 'PRICE_EMA21_DIST', 'ATR', 'VOL', 'VWAP_DIST']


In [3]:
# ============================================================
# COMPREHENSIVE LSTM FEATURE ABLATION
# ============================================================

import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

# Configuration
SEQUENCE_LENGTH = 20
HIDDEN_SIZE = 64
NUM_LAYERS = 2
DROPOUT = 0.3
BATCH_SIZE = 64
EPOCHS = 50
LEARNING_RATE = 0.001
WEIGHT_DECAY = 1e-5

# Results storage
lstm_results = {}

for set_name, feature_list in lstm_feature_sets.items():
    
    print(f"\n{'='*70}")
    print(f"Testing: {set_name} ({len(feature_list)} features)")
    print(f"Features: {feature_list}")
    print(f"{'='*70}")
    
    # Extract features
    X_data = clean_data[feature_list].values
    y_data = y_regression
    
    # Split (chronological)
    n = len(X_data)
    test_start = int(n * 0.8)
    val_start = int(test_start * 0.9)
    
    X_train_raw = X_data[:val_start]
    y_train_raw = y_data[:val_start]
    X_val_raw = X_data[val_start:test_start]
    y_val_raw = y_data[val_start:test_start]
    X_test_raw = X_data[test_start:]
    y_test_raw = y_data[test_start:]
    
    # Create sequence datasets
    train_dataset = SequenceDataset(X_train_raw, y_train_raw, SEQUENCE_LENGTH)
    val_dataset = SequenceDataset(X_val_raw, y_val_raw, SEQUENCE_LENGTH)
    test_dataset = SequenceDataset(X_test_raw, y_test_raw, SEQUENCE_LENGTH)
    
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=False)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)
    
    # Model
    input_size = len(feature_list)
    model = LSTMPredictor(input_size, HIDDEN_SIZE, NUM_LAYERS, DROPOUT).to(device)
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
    criterion = nn.MSELoss()
    
    # Training
    best_val_loss = float('inf')
    best_model_state = None
    
    for epoch in range(EPOCHS):
        # Train
        model.train()
        train_loss = 0.0
        for x_batch, y_batch in train_loader:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            preds = model(x_batch)
            loss = criterion(preds, y_batch)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        train_loss /= len(train_loader)
        
        # Validate
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for x_batch, y_batch in val_loader:
                x_batch, y_batch = x_batch.to(device), y_batch.to(device)
                preds = model(x_batch)
                val_loss += criterion(preds, y_batch).item()
        val_loss /= len(val_loader)
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model_state = model.state_dict().copy()
        
        if (epoch + 1) % 10 == 0:
            print(f"  Epoch {epoch+1}/{EPOCHS} | Train: {train_loss:.4f} | Val: {val_loss:.4f}")
    
    # Test
    model.load_state_dict(best_model_state)
    model.eval()
    
    all_preds, all_targets = [], []
    with torch.no_grad():
        for x_batch, y_batch in test_loader:
            x_batch = x_batch.to(device)
            preds = model(x_batch).cpu().numpy().flatten()
            all_preds.extend(preds)
            all_targets.extend(y_batch.numpy().flatten())
    
    all_preds = np.array(all_preds)
    all_targets = np.array(all_targets)
    
    # Metrics
    test_mse = np.mean((all_targets - all_preds) ** 2)
    test_rmse = np.sqrt(test_mse)
    dir_acc = np.mean((np.sign(all_preds) == np.sign(all_targets)))
    
    lstm_results[set_name] = {
        'n_features': len(feature_list),
        'dir_acc': dir_acc,
        'rmse': test_rmse,
        'mse': test_mse,
        'features': feature_list
    }
    
    print(f"\n  Results: Dir Acc={dir_acc:.2%} | RMSE={test_rmse:.4f}\n")


# ============================================================
# COMPARISON TABLE
# ============================================================

print("\n" + "="*80)
print("LSTM FEATURE ABLATION RESULTS")
print("="*80)
print(f"{'Feature Set':<25} {'Features':>8} {'Dir Acc':>12} {'RMSE':>10} {'vs Baseline':>12}")
print("-"*80)

baseline_acc = 0.5337  # MLP momentum_only

sorted_results = sorted(lstm_results.items(), key=lambda x: x[1]['dir_acc'], reverse=True)

for name, metrics in sorted_results:
    improvement = (metrics['dir_acc'] - baseline_acc) * 100
    marker = " ⭐ BEST" if name == sorted_results[0][0] else ""
    
    print(f"{name:<25} {metrics['n_features']:>8} "
          f"{metrics['dir_acc']:>12.2%} {metrics['rmse']:>10.4f} "
          f"{improvement:+11.2f}pp{marker}")

print("="*80)

# Redundancy Analysis
print("\n" + "="*80)
print("REDUNDANCY ANALYSIS")
print("="*80)

momentum_orig = lstm_results['momentum_original']
momentum_clean = lstm_results['momentum_clean']

print(f"momentum_original (5 feat, with RET_5/15): {momentum_orig['dir_acc']:.2%}")
print(f"momentum_clean (3 feat, no redundancy):    {momentum_clean['dir_acc']:.2%}")

if momentum_clean['dir_acc'] >= momentum_orig['dir_acc']:
    print("\n✅ HYPOTHESIS CONFIRMED: RET_5 and RET_15 were redundant!")
    print("   → Removing them improved or maintained performance with fewer features")
else:
    diff = (momentum_orig['dir_acc'] - momentum_clean['dir_acc']) * 100
    print(f"\n⚠️  Redundant features helped by {diff:.2f}pp")
    print("   → They may encode something LSTM can't learn from RET_1 alone")

print("="*80)


Testing: returns_only (1 features)
Features: ['RET_1']


NameError: name 'clean_data' is not defined